In [6]:
import numpy as np
import pandas as pd

from qnlmodels.cpw import CPW
from qnlmodels.klopfenstein_taper import KlopfensteinTaper
from qnlmodels.rev_kerr_match_impedance import MatchImpedance as match_imp

In [2]:
test_width = 5
s = 3
cpw = CPW(test_width, s, 600, 11.7)
print("Old charateristic impedance = {:.3f}".format(cpw.impedance(cpw.w, cpw.s)))
print("Old width = {:.2f} um".format(cpw.w))
cpw = CPW(cpw.solve_for_impedance(50, 'w'), s, 600, 11.7)
print("New charateristic impedance = {:.3f}".format(cpw.impedance(cpw.w, cpw.s)))
print("New width = {:.3f} um".format(cpw.w))

Old charateristic impedance = 50.450
Old width = 5.00 um
New charateristic impedance = 50.000
New width = 5.157 um


/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlmodels/cpw.py:33: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  K = sp.integrate.quad(lambda t: 1 / np.sqrt( (1 - t**2) * (1 - (k*t)**2) ), 0, 1)
/Users/elias/Desktop/School/Quantum/AQT/TWPA Project/qnlmodels/cpw.py:37: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  return sp.integrate.quad(lambda t: 1 / np.sqrt( (1 - t**2) * (1 - (k*t)**2) ), 0, 1)[0]


In [3]:
#Utils for calulcating Klopfenstein Taper parameters

f = (7.5e9) / (2*np.pi) #signal frequency (Hz)

vp = cpw.phase_velocity(cpw.w, cpw.s)
print("Phase velocity fraction of speed of light: vp / c = {:.4f}".format(vp/3e8))

wavelength =  vp / f
B = 2 * np.pi / wavelength

chip_height = 4.1e-3
number_of_bends = 25
length =  chip_height * number_of_bends #meters

print("Wavelength = {:.3f} meters".format(wavelength))
print("Length = {:.4f} meters".format(length))
print(r'Electrical length =', "{:.3f}".format(length/wavelength))
print("A < {:.3f}".format(B*length))
print("COSH(A) < {:.3f}".format(np.cosh(B*length)))
print("COSH(A) (dB) < {:.3f}".format(20*np.log10(np.cosh(B*length))))

Phase velocity fraction of speed of light: vp / c = 0.3968
Wavelength = 0.100 meters
Length = 0.1025 meters
Electrical length = 1.028
A < 6.458
COSH(A) < 318.929
COSH(A) (dB) < 50.074


In [7]:
Z_line = 50
Z_amp = 86.32

A = int(B*length)

taper = KlopfensteinTaper(A)
Z0_values = taper.get_impedance(Z_line, Z_amp)
print(Z0_values)

TypeError: KlopfensteinTaper.get_impedance() missing 1 required positional argument: 'x_values'

In [ ]:
cpw_widths = [cpw.solve_for_impedance(z0, 'w') for z0 in Z0_values]

In [ ]:
# print(cpw_widths)
print(len(cpw_widths))

x_values = np.linspace(0, 1, len(cpw_widths))

print(cpw_widths)
print(x_values)

11
[5.133155420468808, 4.944660018128904, 4.471881459431433, 3.7024531176896915, 2.8016954310655415, 1.993888289754632, 1.4054093908064218, 1.039551172191768, 0.8408788237697337, 0.749153519410415, 0.7133878353309598]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [ ]:
df = pd.DataFrame([p for p in zip(x_values, cpw_widths)])
df.to_csv('./data/impedance_values.csv', header=["x", "width"], index=False)

In [ ]:
df = pd.DataFrame([p for p in zip(x_values, np.flip(cpw_widths))])
df.to_csv('./data/outgoing_impedance_values.csv', header=["x", "width"], index=False)